In [ ]:
USE_GPU = True
GPU_ID = "3" # the gpu to use for this job
if USE_GPU:
    import os
    os.environ["CUDA_VISIBLE_DEVICES"] = GPU_ID

In [ ]:
import tensorflow as tf
# Suppress tensorflow 1.x deprecation warnings.
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [ ]:
# Initialize logger to view logging output from segmentation tasks
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [ ]:
from google.protobuf import text_format
from ffn.inference import inference
from ffn.inference import inference_pb2

In [ ]:
from fftracer.utils.config import InferenceConfig

# To change inference settings, modify the relevant parameters here.

inference_config = InferenceConfig(
    image="data/test/507727402/507727402_raw.h5:raw", fov_size={"x": 135, "y": 135, "z": 1},
    deltas={"x": 8, "y": 8, "z": 0}, depth=9, image_mean=78, image_stddev=20,
    model_checkpoint_path="training-logs/lr0.001depth9fov135/model.ckpt-3827778",
    model_name="fftracer.training.models.model.FFNTracerModel",
    segmentation_output_dir="results/mozak", move_threshold=0.1,
    min_boundary_dist={"x": 1, "y": 1, "z": 0}
)
config = inference_config.to_string()

In [ ]:
req = inference_pb2.InferenceRequest()
_ = text_format.Parse(config, req)

In [ ]:
runner = inference.Runner()
runner.start(req)
canvas, alignment = runner.make_canvas((0, 0, 0), (1, 9429, 7601)) # sizes in (z, y, x)

In [ ]:
# show the input image data to verify it is valid
from skimage.io import imshow
from matplotlib import pyplot as plt
imshow(canvas.image[0,:,:])
plt.show()


In [ ]:
# Create a single segment, starting from the specified origin point.
canvas.segment_at(
    (0, 4521, 3817),  # seed starting point in (z, y, x)
#     (0, 3801, 4338),  # seed starting point in (z, y, x)
    dynamic_image=inference.DynamicImage(),
    vis_update_every=50,
    vis_fixed_z=True
)

In [ ]:
from ffn.inference.seed import PolicyPeaks2d
canvas.segment_all(seed_policy=PolicyPeaks2d)

In [ ]:
# read the results
import numpy as np
seg_prob = np.load("results/mozak/0/0/seg-0_0_0.prob")['qprob']
print("shape: {}".format(seg_prob.shape))
seg_prob